## Phase 1 : Chargement des Librairies et des Données

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration pour un affichage plus joli des graphiques
sns.set(style="whitegrid")

In [ ]:
# Chargement du jeu de données original
df_original = pd.read_csv('fichier-de-donnees-numeriques-69202f25dea8b267811864.csv')

In [ ]:
# Affichage des 5 premières lignes pour un premier aperçu
print("Shape du DataFrame original:", df_original.shape)
df_original.head()

## Phase 2 : Analyse Exploratoire des Données Brutes

À ce stade, nous n'inspectons que les données sans les modifier. L'objectif est de comprendre leur état initial pour planifier le nettoyage.

### 2.1. Analyse des Valeurs Manquantes


In [ ]:
# Matrice de nullité : une vue d'ensemble des données manquantes
# Les lignes blanches indiquent des données manquantes.
msno.matrix(df_original, figsize=(12, 8), sparkline=False)
plt.title('Matrice de Nullité des Données Originales', fontsize=16)

**Observation :** La matrice confirme visuellement que `historique_credits` et `score_credit` ont de nombreuses valeurs manquantes.

In [ ]:
# Graphique à barres : quantifie le nombre de valeurs non manquantes par colonne
msno.bar(df_original, figsize=(12, 6))
plt.title('Complétude des Données par Colonne', fontsize=16)

In [ ]:
# Heatmap : montre la corrélation de nullité entre les colonnes
# Une valeur proche de 1 signifie que si une donnée manque dans une colonne, elle manque probablement dans l'autre.
# Une valeur proche de -1 signifie que si une donnée est présente, l'autre est probablement absente.
msno.heatmap(df_original, figsize=(10, 8))
plt.title('Heatmap de Corrélation des Données Manquantes', fontsize=16)

### 2.2. Analyse Statistique et Détection des Outliers

In [ ]:
# Statistiques descriptives pour un aperçu numérique
df_original.describe()

**Observation :** Le `min` de `loyer_mensuel` est négatif (-395.25), ce qui est une erreur de donnée évidente.

In [ ]:
# Création de boîtes à moustaches (boxplots) pour chaque variable numérique afin de visualiser les outliers
numerical_cols = df_original.select_dtypes(include=np.number).columns

plt.figure(figsize=(15, 20))
for i, col in enumerate(numerical_cols):
    plt.subplot(5, 2, i + 1)
    sns.boxplot(y=df_original[col])
    plt.title(col)
    plt.tight_layout()

### 2.3. Visualisation des Distributions

In [ ]:
# Histogrammes pour voir la forme de la distribution de chaque variable
plt.figure(figsize=(15, 20))
for i, col in enumerate(numerical_cols):
    plt.subplot(5, 2, i + 1)
    sns.histplot(df_original[col].dropna(), kde=True)
    plt.title(f'Distribution de {col}')
    plt.tight_layout()

## Phase 3 : Nettoyage des Données

Maintenant, nous appliquons les corrections basées sur nos observations. Nous travaillons sur une copie du DataFrame pour conserver l'original.

In [ ]:
df_cleaned = df_original.copy()

### Étape 1 : Supprimer les Colonnes Quasi-Vides

In [ ]:
threshold_col = 50.0
missing_col_percent = df_cleaned.isnull().sum() / len(df_cleaned) * 100
cols_to_drop = missing_col_percent[missing_col_percent > threshold_col].index

df_cleaned.drop(columns=cols_to_drop, inplace=True)
print(f"Colonnes supprimées (plus de {threshold_col}% de valeurs manquantes): {list(cols_to_drop)}")
print("Shape après suppression des colonnes:", df_cleaned.shape)

### Étape 2 : Correction des Outliers

In [ ]:
# Correction du 'loyer_mensuel' négatif en prenant la valeur absolue
df_cleaned['loyer_mensuel'] = df_cleaned['loyer_mensuel'].abs()
print("Valeurs négatives de 'loyer_mensuel' corrigées.")

# Plafonnement (clipping) des outliers pour les colonnes de montant
for col in ['montant_pret', 'revenu_estime_mois']:
    q99 = df_cleaned[col].quantile(0.99)
    df_cleaned[col] = np.where(df_cleaned[col] > q99, q99, df_cleaned[col])
    print(f"Outliers de '{col}' plafonnés à {q99:.2f}.")

### Étape 3 : Imputation des Valeurs Manquantes

In [ ]:
# Imputation avec la médiane pour les colonnes numériques restantes
for col in df_cleaned.select_dtypes(include=np.number).columns:
    if df_cleaned[col].isnull().any():
        median_val = df_cleaned[col].median()
        df_cleaned[col].fillna(median_val, inplace=True)
        print(f"Valeurs manquantes de '{col}' imputées avec la médiane ({median_val:.2f}).")

print("\nVérification des valeurs manquantes après imputation:")
print(df_cleaned.isnull().sum())

## Phase 4 : Analyse Comparative Après Nettoyage

In [ ]:
print("--- Statistiques descriptives - DONNÉES NETTOYÉES ---")
df_cleaned.describe()

In [ ]:
# Comparaison visuelle des distributions avant et après nettoyage
common_cols = list(set(df_original.columns) & set(df_cleaned.columns) & set(numerical_cols))

for col in common_cols:
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    sns.histplot(df_original[col].dropna(), kde=True, color='skyblue')
    plt.title(f'Original - {col}')
    
    plt.subplot(1, 2, 2)
    sns.histplot(df_cleaned[col], kde=True, color='lightcoral')
    plt.title(f'Nettoyé - {col}')
    
    plt.tight_layout()
    plt.show()

## Conclusion

Le processus de nettoyage a permis de traiter les principales anomalies du jeu de données :
- Les colonnes trop vides ont été supprimées.
- Les erreurs de saisie et les valeurs extrêmes ont été corrigées,
    - Toutes les valeurs manquantes ont été imputées de manière robuste.
    
Le jeu de données `df_cleaned` est maintenant plus fiable et prêt à être utilisé pour l'entraînement de modèles